# Sprint15課題 ディープラーニングフレームワーク2

## 【問題1】公式Exampleを分担して実行

TensorFLowの公式Exampleを分担して実行してください。

以下の中から1人ひとつ選び実行し、その結果を簡単に発表してください。

research

定番のモデルから最新のモデルまで多様なコードが公開されています。

[models/research at master · tensorflow/models](https://github.com/tensorflow/models/tree/master/research)

tutorials

TensorFLowのチュートリアルとして用意された簡単なモデルが含まれています。

[models/tutorials at master · tensorflow/models](https://github.com/tensorflow/models/tree/master/tutorials)

**前準備**

In [0]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [0]:
# Import TensorFlow
!pip install tf-nightly-gpu
import tensorflow as tf
import tensorflow_datasets as tfds

import os

In [0]:
#MNISTデータを読み込み
datasets, ds_info = tfds.load(name='mnist', with_info=True, as_supervised=True)
mnist_train, mnist_test = datasets['train'], datasets['test']

W0531 10:05:47.202017 139959173076864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow_datasets/core/dataset_utils.py:252: DatasetV1.output_shapes (from tensorflow.python.data.ops.dataset_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.compat.v1.data.get_output_shapes(dataset)`.


In [0]:
#レプリカの作成
strategy = tf.distribute.MirroredStrategy()

W0531 10:08:55.921426 139959173076864 cross_device_ops.py:1168] There is non-GPU devices in `tf.distribute.Strategy`, not using nccl allreduce.


In [0]:
#GPUデバイスの数
print ('Number of devices: {}'.format(strategy.num_replicas_in_sync))

Number of devices: 1


In [0]:
#学習用データと検証用データの合計
num_train_examples = ds_info.splits['train'].num_examples
num_test_examples = ds_info.splits['test'].num_examples

BUFFER_SIZE = 10000

BATCH_SIZE_PER_REPLICA = 64
BATCH_SIZE = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

In [0]:
def scale(image, label):
  '''
  0~255のピクセルを0~1に正規化する
  '''
  image = tf.cast(image, tf.float32)
  image /= 255

  return image, label

In [0]:
#正規化したデータをシャッフルし、バッチ数ごとに分ける
train_dataset = mnist_train.map(scale).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
eval_dataset = mnist_test.map(scale).batch(BATCH_SIZE)

In [0]:
#モデルを作成
with strategy.scope():
  model = tf.keras.Sequential([
      #畳み込み層(出力チャンネル数、フィルタサイズ, 活性化関数, 入力サイズ)
      tf.keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)),
      
      #プーリング層
      tf.keras.layers.MaxPooling2D(),
      
      #平滑化
      tf.keras.layers.Flatten(),
      
      #全結合層
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
  ])

  #コンパイル
  model.compile(loss='sparse_categorical_crossentropy',
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])

**コールバックの定義**

In [0]:
#チェックポイントのファイル名
checkpoint_dir = './training_checkpoints'

#チェックポイントのファイル名とパス名をくっつける
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

In [0]:
def decay(epoch):
  '''
  学習率を下げる機能
  '''
  if epoch < 3:
    return 1e-3
  elif epoch >= 3 and epoch < 7:
    return 1e-4
  else:
    return 1e-5

In [0]:
class PrintLR(tf.keras.callbacks.Callback):
  '''
  エポックごとに学習率を出力するクラス
  '''
  def on_epoch_end(self, epoch, logs=None):
    print ('\nLearning rate for epoch {} is {}'.format(
        epoch + 1, tf.keras.backend.get_value(model.optimizer.lr)))

In [0]:
callbacks = [
    #TensorfBoardによって解析されたログファイルを保存するディレクトリのパス
    tf.keras.callbacks.TensorBoard(log_dir='./logs'),
    
    #エポックごとにモデルを保存
    #fillpath : モデルファイルを保存するパス
    #save_weights_only : モデルの重みを保存
    tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,
                                       save_weights_only=True),
    
    #学習率のスケジューラ(エポックのインデックスを入力とし，新しい学習率を返す)
    tf.keras.callbacks.LearningRateScheduler(decay),
    PrintLR()
]

**学習、推定**

In [0]:
#学習
model.fit(train_dataset, epochs=10, callbacks=callbacks)

Train on 938 steps
Epoch 1/10
937/938 [============================>.] - ETA: 0s - loss: 0.2019 - acc: 0.9414
Learning rate for epoch 1 is 0.0010000000474974513
938/938 [==============================] - 31s 33ms/step - loss: 0.2018 - acc: 0.9414
Epoch 2/10
936/938 [============================>.] - ETA: 0s - loss: 0.0683 - acc: 0.9804
Learning rate for epoch 2 is 0.0010000000474974513
938/938 [==============================] - 30s 32ms/step - loss: 0.0683 - acc: 0.9804
Epoch 3/10
937/938 [============================>.] - ETA: 0s - loss: 0.0486 - acc: 0.9856
Learning rate for epoch 3 is 0.0010000000474974513
938/938 [==============================] - 31s 33ms/step - loss: 0.0486 - acc: 0.9855
Epoch 4/10
937/938 [============================>.] - ETA: 0s - loss: 0.0271 - acc: 0.9923
Learning rate for epoch 4 is 9.999999747378752e-05
938/938 [==============================] - 31s 33ms/step - loss: 0.0271 - acc: 0.9923
Epoch 5/10
937/938 [============================>.] - ETA: 0s - loss:

In [0]:
#チェックポイントのディレクトリ
!ls {checkpoint_dir}

checkpoint		     ckpt_5.data-00000-of-00002
ckpt_10.data-00000-of-00002  ckpt_5.data-00001-of-00002
ckpt_10.data-00001-of-00002  ckpt_5.index
ckpt_10.index		     ckpt_6.data-00000-of-00002
ckpt_1.data-00000-of-00002   ckpt_6.data-00001-of-00002
ckpt_1.data-00001-of-00002   ckpt_6.index
ckpt_1.index		     ckpt_7.data-00000-of-00002
ckpt_2.data-00000-of-00002   ckpt_7.data-00001-of-00002
ckpt_2.data-00001-of-00002   ckpt_7.index
ckpt_2.index		     ckpt_8.data-00000-of-00002
ckpt_3.data-00000-of-00002   ckpt_8.data-00001-of-00002
ckpt_3.data-00001-of-00002   ckpt_8.index
ckpt_3.index		     ckpt_9.data-00000-of-00002
ckpt_4.data-00000-of-00002   ckpt_9.data-00001-of-00002
ckpt_4.data-00001-of-00002   ckpt_9.index
ckpt_4.index


In [0]:
#最新のチェックポイントの重みを呼び出す
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

#testデータの評価
eval_loss, eval_acc = model.evaluate(eval_dataset)
print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 2s 12ms/step - loss: 0.0368 - acc: 0.9878
Eval loss: 0.03682094065926281, Eval Accuracy: 0.9878000020980835


ターミナルでTensorBoardのログを見る

```
$ tensorboard --logdir=path/to/log-directory
```

In [0]:
#TensorBoardのログ
!ls -sh ./logs

total 424M
107M events.out.tfevents.1559289829.a5c9cce260bc
4.0K events.out.tfevents.1559289831.a5c9cce260bc.profile-empty
212M events.out.tfevents.1559291625.a5c9cce260bc
107M events.out.tfevents.1559299757.a5c9cce260bc
4.0K plugins


**Modelを保存、エクスポート**

In [0]:
#マウント
from google.colab import drive
drive.mount('./gdrive')

Drive already mounted at ./gdrive; to attempt to forcibly remount, call drive.mount("./gdrive", force_remount=True).


In [0]:
!ls gdrive

'My Drive'


In [0]:
path = './gdrive/My Drive/saved_model/'

In [0]:
#モデルをMyDriveに保存
tf.keras.experimental.export_saved_model(model, path)

W0531 11:15:35.268236 139959173076864 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0531 11:15:35.270257 139959173076864 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0531 11:15:36.181991 139959173076864 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from tensorflow.python.saved_model.uti

In [0]:
#モデルをロード
unreplicated_model = tf.keras.experimental.load_from_saved_model(path)

#コンパイル
unreplicated_model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'])

#評価
eval_loss, eval_acc = unreplicated_model.evaluate(eval_dataset)
print ('Eval loss: {}, Eval Accuracy: {}'.format(eval_loss, eval_acc))

157/157 [==============================] - 2s 15ms/step - loss: 0.0368 - acc: 0.9878
Eval loss: 0.03682094065926281, Eval Accuracy: 0.9878000020980835
